In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00


In [2]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, LSTM, Dense, GlobalMaxPooling1D

In [66]:
data = pd.read_csv('dataset1.csv')
print(data)

          id                                               text  target
0          0  Communal violence in Bhainsa, Telangana. "Ston...       1
1          1  Telangana: Section 144 has been imposed in Bha...       1
2          2  Arsonist sets cars ablaze at dealership https:...       1
3          3  Arsonist sets cars ablaze at dealership https:...       1
4          4  "Lord Jesus, your love brings freedom and pard...       0
...      ...                                                ...     ...
11365  11365  Media should have warned us well in advance. T...       0
11366  11366  i feel directly attacked ðŸ’€ i consider moonb...       0
11367  11367  i feel directly attacked ðŸ’€ i consider moonb...       0
11368  11368  ok who remember "outcast" nd the "dora" au?? T...       0
11369  11369     Jake Corway wrecked while running 14th at IRP.       1

[11370 rows x 3 columns]


In [67]:
data['text'] = data['text'].str.replace('[^\w\s]','') 
data['text'] = data['text'].str.replace('\d+','')  
data['text'] = data['text'].str.strip()

<ipython-input-67-6be140546912>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('[^\w\s]','')
<ipython-input-67-6be140546912>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('\d+','')


In [68]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)


In [69]:
print("Training Data:")
print(train_data)

print("Testing Data:")
print(test_data)

Training Data:
          id                                               text  target
9313    9313  Theres a wonderful  photo view of the inside o...       0
8369    8369  The warned everybody FOUR years ago about this...       0
2762    2762  Hello Eric  A social media serye where Lola Ca...       0
1683    1683  How many burning buildings have you run into t...       0
2487    2487  Sgt David Edward Lloyd One of THE FEW httpstco...       0
...      ...                                                ...     ...
11284  11284  After causing the wreck theyre like httpstcokZ...       0
5191    5191  Letâs get the Starboy to k subscribers Before ...       0
5390    5390  Been a long day of storm surveys and one of ou...       0
860      860  As we work on the next one this was what leadi...       0
7270    7270  Im not supporting the colony narrative but it ...       0

[10233 rows x 3 columns]
Testing Data:
          id                                               text  target
3495    3

In [70]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [71]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
import torch

In [73]:
def bert_encode(texts, tokenizer, max_len=512):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    return input_ids, attention_masks
    
train_input_ids, train_attention_masks = bert_encode(train_data['text'].values, tokenizer)
test_input_ids, test_attention_masks = bert_encode(test_data['text'].values, tokenizer)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [74]:
model = Sequential()
model.add(Embedding(input_dim=tokenizer.vocab_size, output_dim=100, input_length=1024))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(LSTM(units=64, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=1, activation='sigmoid'))


In [75]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
import tensorflow as tf

In [77]:
from tensorflow.keras.layers import Concatenate

train_input_ids_tf = tf.convert_to_tensor(train_input_ids.numpy())
train_attention_masks_tf = tf.convert_to_tensor(train_attention_masks.numpy())

concat_layer = Concatenate()([train_input_ids_tf, train_attention_masks_tf])


In [78]:
model.fit(
    concat_layer.numpy(),
    train_data['target'].values,
    epochs=5,
    batch_size=32
)


Epoch 1/5
320/320 [==============================] - 285s 879ms/step - loss: 0.4051 - accuracy: 0.8369
Epoch 2/5
320/320 [==============================] - 281s 879ms/step - loss: 0.2179 - accuracy: 0.9248
Epoch 3/5
320/320 [==============================] - 282s 882ms/step - loss: 0.1291 - accuracy: 0.9604
Epoch 4/5
320/320 [==============================] - 280s 874ms/step - loss: 0.0769 - accuracy: 0.9780
Epoch 5/5
320/320 [==============================] - 282s 881ms/step - loss: 0.0505 - accuracy: 0.9871


In [79]:
test_input_ids_tf = tf.convert_to_tensor(test_input_ids.numpy())
test_attention_masks_tf = tf.convert_to_tensor(test_attention_masks.numpy())

concat_layer = Concatenate()([test_input_ids_tf, test_attention_masks_tf])
test_pred = model.predict(concat_layer.numpy())
test_pred = np.round(test_pred).flatten()
test_true = test_data['target'].values

36/36 [==============================] - 6s 153ms/step


In [80]:
accuracy = accuracy_score(test_true, test_pred)
precision = precision_score(test_true, test_pred)
recall = recall_score(test_true, test_pred)
f1 = f1_score(test_true, test_pred)

In [81]:
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 0.9023746701846965
Precision: 0.75625
Recall: 0.6269430051813472
F1 Score: 0.6855524079320112
